## Scaling Alternating Least Squares Using Apache SystemML

This notebook uses some material from the the original notebook in the SystemML repo:
https://raw.githubusercontent.com/apache/systemml/master/samples/jupyter-notebooks/ALS_python_demo.ipynb


Recommendation systems based on Alternating Least Squares (ALS) algorithm have gained popularity in recent years because, in general, they perform better as compared to content based approaches.




ALS is a matrix factorization algorithm, where a user-item matrix is factorized into two low-rank non-orthogonal matrices:

$${\bf R} = {\bf U} {\bf M}$$

The elements, $r_{ij}$, of matrix $R$ can represent, for example, ratings assigned to the $j$th movie by the $i$th user.

This matrix factorization assumes that each user can be described by $K$ latent features. Similarly, each item/movie can also be represented by $K$ latent features. The user rating of a particular movie can thus be approximated by the product of two $K$-dimensional vectors:

$$r_{ij} = {\bf u}_i^T {\bf m}_j$$

The vectors ${\bf u}_i$ are rows of $U$ and ${\bf m}_j$'s are columns of $M$. These can be learned by minimizing the cost function:

$$f(U, M) = \sum_{i,j} \left( r_{ij} - {\bf u}_i^T {\bf m}_j \right)^2 = \| R - UM \|^2$$

## Understanding the Model as a Latent Feature Model

The "score" or label of a particular movie/user pair is
$$r_{ij} = {\bf u}_i^T {\bf m}_j$$
for the $i$th user and $j$th movie.  One way to think of the rank is in terms of the *movie score* only as a linear model:

$$r_{ij} = \sum_{k=1}^{K} \alpha_k m_{j,k}$$

where the movie vector is now represented as feature vector with $K$ features, and the user vector is simply a set of learned parameters from the data.  We define how many features this internal representation will have, but the resultant output is really the only thing we are concerned with.  These "features" are simply learned during the process of fitting the objective function, and may or may not be mapped to an actual, explainable feature set that we might understand.  As such, they are referred to as *latent features*.  In this sence, these latent features are akin to clusters that you might see in an unsupervised learning model, which makes this model an interesting hybrid model.

Yet another way to think of the same score is the *user score* as a linear model:

$$r_{ij} = \sum_{k=1}^{K} \beta_k u_{i,k}$$

Now, the features are related to the user vector, and the parameters are learned weights.  In fact, the training process is an alternation between solving for the weights of the *movie vector* while holding the user features constant, and the weights for the *user vector* holding the movie features constant.  The numerical solution of the objective is obtained through this *alternating least squares* approach.  T

If this model were not so popular and powerful, it might otherwise be considered an interesting peculiarity of machine learning.  It is however, the most widely used machine learning algorithm in e-commerce, and is particularly effective for *recommender sytems*, whereby a list of recommended items can be generated for new users based on the preferences of similar previous users.

In [1]:
sc

Spark local-1543261122307: Some(http://rally1.fyre.ibm.com:4040)

In [2]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

Since we are using scala, we will leverage the convenience of a `case class` to package some of our parameter settings. 

In [14]:
case class Parameters(filename: String, rank: Int = 5, numDSIterations: Int = 5, 
                  lambda: Double = 0.1, trainFraction: Double = 0.9, 
                  trainSeed: Long = 0L, splitSeed: Int = 0) {
                      //JNDB TODO:  need to be typed.
                      val testFraction = 1 - trainFraction
                      val data = sc.textFile(filename)
                      
                      val Array(dataTrain, dataTest) = data.randomSplit(Array(trainFraction, testFraction), splitSeed)
                      dataTrain.cache
                      dataTest.cache
                      val (trainNum, testNum) = (dataTrain.count, dataTest.count)
                      val totalNum = trainNum - testNum
                      val display = {
                        "filename: " + filename  + "\n" +
                        "rank: " + rank.toString + "\n" + 
                        "lambda: " + lambda.toString + "\n" + 
                        "# DS iterations: " + numDSIterations.toString + "\n" + 
                        "trainingFraction: " + trainFraction.toString  + "\n" +
                        "training random seed: " + trainSeed.toString  + "\n" +
                        "splitting random seed: " + splitSeed.toString + "\n" + 
                        "# (training, test, total) points:  " + (testNum, trainNum, totalNum) + "\n"  
                    }
  }

defined class Parameters


Now we can create a single case class with all of the information required to run our ALS prediction.

In [29]:
val p = Parameters(filename = "sample_movielens_ratings.txt", 
                              rank = 1500, 
                              numDSIterations = 15, 
                              lambda = 0.1)

p = Parameters(sample_movielens_ratings.txt,1500,15,0.1,0.9,0,0)


Parameters(sample_movielens_ratings.txt,1500,15,0.1,0.9,0,0)

Let's create a function that will parse our dataset.

In [24]:
case class RatingML(userId: Int, movieId: Int, rating: Float, timestamp: Long)

def parseRating(str: String): RatingML = {
    val fields = str.split("::")
    assert(fields.size == 4)
    RatingML(fields(0).toInt, fields(1).toInt, fields(2).toFloat, fields(3).toLong)
}

parseRating: (str: String)RatingML


In our `Parameters` class, we split our data into training and test sets.  Lets parse those raw ratings into a proper `DataFrame` so we can use the ML Pipelines API.

In [ ]:
val train = p.dataTrain.map(parseRating).toDF()
val test = p.dataTest.map(parseRating).toDF()

###  Regularized ALS

In this notebook, we'll implement ALS algorithm with _weighted-$\lambda$-regularization_ formulated by [Zhou _et_. _al_](https://liuxiaofei.com.cn/blog/wp-content/uploads/2014/01/netflix_aaim08submitted.pdf). The cost function with such regularization is:

$$f(U, M) = \sum_{i,j} I_{ij}\left( r_{ij} - {\bf u}_i^T {\bf m}_j \right)^2 + \lambda \left( \sum_i n_{u_i} \| {\bf u}\|_i^2 + \sum_j n_{m_j} \|{\bf m}\|_j^2 \right)$$


Here, $\lambda$ is the usual regularization parameter. $n_{u_i}$ and $n_{m_j}$ represent the number of ratings of user $i$ and movie $j$ respectively. $I_{ij}$ is an indicator variable such that $I_{ij} = 1$ if $r_{ij}$ exists and $I_{ij} = 0$ otherwise.

If we fix ${\bf m}_j$, we can determine ${\bf u}_i$ by solving a regularized least squares problem:

$$ \frac{1}{2} \frac{\partial f}{\partial {\bf u}_i} = 0$$

This gives the following matrix equation:

$$\left(M \text{diag}({\bf I}_i^T) M^{T} + \lambda n_{u_i} E\right) {\bf u}_i = M {\bf r}_i^T$$

Here ${\bf r}_i^T$ is the $i$th row of $R$. Similarly, ${\bf I}_i$ the $i$th row of the matrix $I = [I_{ij}]$.

Here is what the first few entries look like. We have a sparse mapping of users and movies to ratings.  The goal of the ALS algorithm is to predict the rating of a user/movie pairing that does not exist in the training dataset.  This is accomplished by creating a set of latent features (defined by the "rank" parameter here) from the training dataset and using those latent features to predict on new data.    

In [35]:
train.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
+------+-------+------+----------+
only showing top 5 rows



In [30]:
val t0 = System.nanoTime
  // Build the recommendation model using ALS on the training data
val als = new ALS 
als.
  setMaxIter(p.numDSIterations).
  setRank(p.rank).
  setRegParam(p.lambda).
  setUserCol("userId").
  setItemCol("movieId").
  setRatingCol("rating")
val model = als.fit(train)
     
  // Evaluate the model by computing the RMSE on the test data
val predictions = model.transform(test)
predictions.createOrReplaceTempView("predictions")
val predictionsWithDeltas = spark.sql("select userID, movieID, rating, round(prediction), (rating - prediction) as delta from predictions")
    
val t1 = System.nanoTime

println("time to compute (in seconds): "  + Math.round((t1-t0) / 1.0e6) / 1.0e3 )
println("time to compute (in minutes): "  + Math.round((t1-t0) / 60.0 / 1.0e6) / 1.0e3 )

time to compute (in seconds): 37.265
time to compute (in minutes): 0.621


train = [userId: int, movieId: int ... 2 more fields]
test = [userId: int, movieId: int ... 2 more fields]
t0 = 3282565836121988
als = als_6cdf7ac2d743
model = als_6cdf7ac2d743
predictions = [userId: int, movieId: int ... 3 more fields]
predictionsWithDeltas = [userID: int, movieID: int ... 3 more fields]
t1 = 3282603101118972


3282603101118972

In [32]:
predictionsWithDeltas.show

+------+-------+------+--------------------+------------+
|userID|movieID|rating|round(prediction, 0)|       delta|
+------+-------+------+--------------------+------------+
|    26|     31|   1.0|                 0.0|  0.52631253|
|     5|     31|   1.0|                 1.0| -0.03673148|
|    25|     31|   2.0|                 2.0| -0.23639536|
|    15|     85|   1.0|                 1.0| -0.45566332|
|    21|     85|   3.0|                 3.0|  0.37177014|
|    28|     65|   1.0|                 1.0| -0.10253787|
|    23|     65|   5.0|                 2.0|   2.9174232|
|    24|     65|   1.0|                 1.0|-0.029762268|
|    27|     78|   1.0|                 1.0|  0.23632115|
|     4|     78|   1.0|                 1.0|  0.14433408|
|    21|     81|   1.0|                 1.0| -0.08326793|
|    18|     28|   5.0|                 1.0|   3.9147174|
|     6|     26|   1.0|                 1.0| -0.22810721|
|    15|     26|   3.0|                 1.0|   1.7759048|
|    26|     2

In [33]:
val count = predictionsWithDeltas.count
// RMSE (root mean squared error) is computed in the usual way
val rmse = Math.sqrt(predictionsWithDeltas.select("delta").map(x => Math.pow(x(0).asInstanceOf[Float], 2)).reduce(_+_)/count) 
// A true positive is defined as a prediction that lands within 0.5 of the correct prediction (using delta column)
val truePositives =  predictionsWithDeltas.select("delta").filter(x=>Math.abs(x(0).asInstanceOf[Float]) <= 0.5).count 
val falsePositives = count - truePositives
val accuracy = truePositives.toDouble/count.toDouble

println(p.display)
println("# test points: " + count)
println("rmse: " + rmse)
println("correct predictions: " + truePositives)
println("incorrect predictions:" + falsePositives)
println("# predictions: " + count)
println("accuracy: " + accuracy)

//println("time to compute (in seconds): "  + Math.round((t1-t0) / 1.0e6) / 1.0e3 )
//println("time to compute (in minutes): "  + Math.round((t1-t0) / 60.0 / 1.0e6) / 1.0e3 )

filename: sample_movielens_ratings.txt
rank: 1500
lambda: 0.1
# DS iterations: 15
trainingFraction: 0.9
training random seed: 0
splitting random seed: 0
# (training, test, total) points:  (154,1347,1193)

# test points: 154
rmse: 1.0696576978925214
correct predictions: 83
incorrect predictions:71
# predictions: 154
accuracy: 0.538961038961039


count = 154
rmse = 1.0696576978925214
truePositives = 83
falsePositives = 71
accuracy = 0.538961038961039


0.538961038961039